In [1]:
# On importe les librairies nécessaires à notre description
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.offline import plot
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import shapiro, wilcoxon, pearsonr, bartlett, ttest_ind

In [2]:
# On charge les données que l'on va utiliser
# Ici, on s'intéresse aux différences entre Femmes et Hommes
filepath = 'coviprev-age.csv'
df = pd.read_csv(filepath, sep=';')
df.head()

,age,semaine,anxiete,anxiete_inf,anxiete_sup,depression,depression_inf,depression_sup,pbsommeil,pbsommeil_inf,...,nbmoy4mesHyg_sup,dist1m,dist1m_inf,dist1m_sup,portmasque,portmasque_inf,portmasque_sup,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,18-24 ans,Vague 1 : 23-25 mars,"33,1","26,8","40,0",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25-34 ans,Vague 1 : 23-25 mars,"37,5","32,2","43,1",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35-49 ans,Vague 1 : 23-25 mars,"32,0","28,1","36,2",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50-64 ans,Vague 1 : 23-25 mars,"21,4","18,0","25,2",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65 ans et plus,Vague 1 : 23-25 mars,"17,0","13,9","20,7",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.drop(["Unnamed: 29", "Unnamed: 30", "Unnamed: 31"], axis=1, inplace=True)

In [4]:
df['semaine'] = df['semaine'].apply(lambda x:str(x))

In [5]:
df['vague'] = df["semaine"].apply(lambda x: x.split(':')[0])
#df['Semaine'] = df["semaine"].apply(lambda x: x.split(':')[1])

In [6]:
# On sélectionne les variables qui seront intéressantes pour cette partie de l'analyse
data = df[["age",'vague', 'semaine', 'anxiete', 'depression','pbsommeil']]
del df

In [7]:
data.head()

,age,vague,semaine,anxiete,depression,pbsommeil
0,18-24 ans,Vague 1,Vague 1 : 23-25 mars,"33,1",NaN,NaN
1,25-34 ans,Vague 1,Vague 1 : 23-25 mars,"37,5",NaN,NaN
2,35-49 ans,Vague 1,Vague 1 : 23-25 mars,"32,0",NaN,NaN
3,50-64 ans,Vague 1,Vague 1 : 23-25 mars,"21,4",NaN,NaN
4,65 ans et plus,Vague 1,Vague 1 : 23-25 mars,"17,0",NaN,NaN


In [8]:
# On remarque que nos valeurs ont le type "object"
# On modifie donc les variables "anxiete", "depression" et "pbsommeil"
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.to_numeric(data[col].str.replace(',', '.'), errors='ignore')

In [9]:
data.head()

,age,vague,semaine,anxiete,depression,pbsommeil
0,18-24 ans,Vague 1,Vague 1 : 23-25 mars,33.1,NaN,NaN
1,25-34 ans,Vague 1,Vague 1 : 23-25 mars,37.5,NaN,NaN
2,35-49 ans,Vague 1,Vague 1 : 23-25 mars,32.0,NaN,NaN
3,50-64 ans,Vague 1,Vague 1 : 23-25 mars,21.4,NaN,NaN
4,65 ans et plus,Vague 1,Vague 1 : 23-25 mars,17.0,NaN,NaN


In [10]:
data.isnull().mean()

age           0.090909
vague         0.000000
semaine       0.000000
anxiete       0.090909
depression    0.147727
pbsommeil     0.147727
dtype: float64

In [11]:
data.dropna(inplace=True)

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

age = data.age.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="checklist",
        options=[{"label": x, "value": x} 
                 for x in age],
        value=age[3:],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="line-chart"),
])

@app.callback(
    Output("line-chart", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(age):
    mask = data.age.isin(age)
    fig = px.line(data[mask], 
        x="vague", y="anxiete", color='age',
                 title="Evolution de l'anxiété au cours du temps en fonction de l'âge")
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
age = data.age.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="checklist",
        options=[{"label": x, "value": x} 
                 for x in age],
        value=age[3:],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="line-chart"),
])

@app.callback(
    Output("line-chart", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(age):
    mask = data.age.isin(age)
    fig = px.line(data[mask], 
        x="vague", y="depression", color='age',
                 title="Evolution de la dépression au cours du temps en fonction de l'âge")
    return fig

app.run_server(debug=True, use_reloader=False)

In [ ]:
age = data.age.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="checklist",
        options=[{"label": x, "value": x} 
                 for x in age],
        value=age[3:],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="line-chart"),
])

@app.callback(
    Output("line-chart", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(age):
    mask = data.age.isin(age)
    fig = px.line(data[mask], 
        x="vague", y="pbsommeil", color='age',
                 title="Evolution des troubles du sommeil au cours du temps en fonction de l'âge")
    return fig

app.run_server(debug=True, use_reloader=False)